In [ ]:
import os
import sqlite3
import pandas as pd
import datetime as dt

pd.options.plotting.backend = "plotly"

path = '../data/database.db'
conn = sqlite3.connect(path)
cursor = conn.cursor()

#get all table names form the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
REQUESTS
sqlite_sequence
CPI
Population
Flux
Employment
GDP
Population_by_municipality
Interest


In [111]:
query = """
SELECT "Consumer price index Index" FROM CPI
WHERE "Month" = (SELECT MAX("Month") FROM CPI)
"""
cursor.execute(query)
CPI = cursor.fetchone()[0]
CPI

649.7

In [ ]:

query = """
SELECT "{}" FROM Population
WHERE SEX == "Total" AND AGE == "Total"
""".format(dt.datetime.now().year)

cursor.execute(query)
population = cursor.fetchone()[0]
population


389444

In [24]:

query = """
SELECT "TOTAL TOTAL" FROM Flux
WHERE QUARTER = (SELECT MAX(QUARTER) FROM Flux)
AND EVENT == "Net migration"
"""

cursor.execute(query)
net_migration = cursor.fetchone()[0]
net_migration



290

In [ ]:

query = """
SELECT "Economic Activity", "2024Q4" From Employment
WHERE "Value unit" == 'Persons'
AND "Employment" == "Total employment"
AND "Economic Activity" != "Total - All activities"
"""
pieFrame = pd.read_sql_query(query, conn)

#make pie chart with plotly
import plotly.express as px
fig = px.pie(pieFrame, values='2024Q4', names='Economic activity')
fig.update_traces(showlegend=False)
fig.update_layout(title_font_size=20, title_x=0.5, title_y=0.95, title_text="Total Employment by Economic Activity in 2024Q4")
# pieFrame
fig


In [ ]:
qry = """
SELECT "2024Q4"
FROM GDP
WHERE "Value unit" == "Current prices" AND
Category == "8. Gross Domestic Product"
"""

cursor.execute(qry)
GDP = cursor.fetchone()[0]
GDP

'1178802'

In [96]:
query = "Select * from Population_by_municipality"
df = pd.read_sql_query(query, conn)

df.query("Quarter == '2024Q4' & ~Municipality.isin(['Total', 'Capital region'])")


query = """
SELECT TOTAL, MALES, FEMALES, "Non-Binary/Other", "Icelandic citizens", "Foreign citizens"
FROM Population_by_municipality
WHERE Quarter == '2024Q4' AND Municipality == 'Total'
"""

dictionary = pd.read_sql_query(query, conn)
dictionary = dictionary.to_dict(orient='records')[0]
data = dictionary

In [116]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data
data = {
    'Total': 389450,
    'Males': 199620,
    'Females': 189620,
    'Non-binary/Other': 200,
    'Icelandic citizens': 321870,
    'Foreign citizens': 67580
}

# Create subplots with 3 domains (left, center, right)
fig = make_subplots(rows=3, cols=2,
                    specs=[[{'type':'domain'},{'type':'domain'}], 
                           [{'type':'domain'},{'type':'domain'}],
                           [{'type':'domain'},{'type':'domain'}]],
                    column_widths=[0.5,0.5])

fig.add_trace(go.Pie(
    labels=['Males', 'Females', 'Non-binary/Other'],
    values=[data['Males'], data['Females'], int(data['Non-binary/Other'])],
    name='Sex',
    hole=0.4
), row=1, col=1)

fig.add_trace(go.Pie(
    labels=['Total'],
    values=[data['Total']],
    name='Total',
    hole=0.6,
    marker_colors=['lightgrey'],
    textinfo='label+value'
), row=2, col=2)

fig.add_trace(go.Pie(
    labels=['Icelandic citizens', 'Foreign citizens'],
    values=[data['Icelandic citizens'], int(data['Foreign citizens'])],
    name='Citizenship',
    hole=0.4
), row=3, col=1)

fig.update_layout(
    title_text="Iceland Population Breakdown (2025)",
    title_x=0.5,
    width = 450,
    height = 800,
    showlegend=False,
)

fig.show()

